In [1]:
import requests
from collections import defaultdict
import time

In [2]:

UNISWAP_V3_SUBGRAPH_URL = "https://gateway.thegraph.com/api/214ad8fea4f34456c35952f32963b967/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

def fetch_october_2023_tokens_from_pools():
    query = """
    {
      pools(
        first: 1000,
        orderBy: createdAtTimestamp,
        orderDirection: asc,
        where: {
          createdAtTimestamp_gte: 1706745600,
          createdAtTimestamp_lt:  1709164800
        }
      ) {
        id
        createdAtTimestamp
        token0 {
          id
          symbol
          name
          decimals
        }
        token1 {
          id
          symbol
          name
          decimals
        }
      }
    }
    """

    response = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={"query": query})
    data = response.json()

    if "errors" in data:
        print("GraphQL error:", data["errors"])
        return []

    tokens = {}
    for pool in data["data"]["pools"]:
        for token_key in ["token0", "token1"]:
            token = pool[token_key]
            tokens[token["id"]] = {
                "address": token["id"],
                "symbol": token["symbol"],
                "name": token["name"],
                "decimals": int(token["decimals"])
            }

    print(f"✅ Found {len(tokens)} unique ERC-20 tokens in pools created feb 2024.")
    return list(tokens.values())

tokens_october = fetch_october_2023_tokens_from_pools()


✅ Found 950 unique ERC-20 tokens in pools created feb 2024.


In [3]:
API_KEY = "214ad8fea4f34456c35952f32963b967"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
SUBGRAPH_URL = "https://gateway.thegraph.com/api/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

# timestamps
START_TS = 1706745600  # fev 1, 2024
END_TS = 1709164800    # feb 29, 2024

def run_query(query):
    response = requests.post(SUBGRAPH_URL, json={"query": query}, headers=HEADERS)
    if response.status_code == 200:
        resp_json = response.json()
        if "errors" in resp_json:
            raise Exception(f"GraphQL errors: {resp_json['errors']}")
        return resp_json["data"]
    else:
        raise Exception(f"Query failed with status code {response.status_code}: {response.text}")

def fetch_pools_created_october():
    pools = []
    skip = 0
    batch_size = 1000

    while True:
        query = f"""
        {{
          pools(
            first: {batch_size},
            skip: {skip},
            orderBy: createdAtTimestamp,
            orderDirection: asc,
            where: {{
              createdAtTimestamp_gte: {START_TS},
              createdAtTimestamp_lt: {END_TS}
            }}
          ) {{
            id
            createdAtTimestamp
            token0 {{
              id
              symbol
              name
              decimals
            }}
            token1 {{
              id
              symbol
              name
              decimals
            }}
          }}
        }}
        """
        data = run_query(query)
        batch = data["pools"]
        pools.extend(batch)
        print(f"Fetched {len(batch)} pools, total so far: {len(pools)}")
        if len(batch) < batch_size:
            break
        skip += batch_size
        time.sleep(0.3)  # be gentle to API

    return pools

def extract_unique_tokens_from_pools(pools):
    tokens = {}
    for pool in pools:
        for token_key in ["token0", "token1"]:
            token = pool[token_key]
            tokens[token["id"]] = {
                "symbol": token["symbol"],
                "name": token["name"],
                "decimals": int(token["decimals"]),
                "address": token["id"],
            }
    return tokens

def fetch_pool_day_data(pool_ids):
    pool_volumes = defaultdict(float)
    batch_size = 50

    # Fetch daily volumes for each pool for October
    for i in range(0, len(pool_ids), batch_size):
        batch_ids = pool_ids[i : i + batch_size]
        ids_string = ','.join(f'"{pid}"' for pid in batch_ids)

        query = f"""
        {{
          poolDayDatas(
            where: {{
              pool_in: [{ids_string}],
              date_gte: {START_TS},
              date_lt: {END_TS}
            }},
            first: 1000
          ) {{
            pool {{
              id
              token0 {{
                id
                symbol
              }}
              token1 {{
                id
                symbol
              }}
            }}
            volumeUSD
            date
          }}
        }}
        """
        data = run_query(query)
        day_data = data["poolDayDatas"]

        for record in day_data:
            pool = record["pool"]
            vol = float(record["volumeUSD"])
            # Assign half volume to token0 and half to token1 (simplification)
            pool_volumes[pool["token0"]["id"]] += vol / 2
            pool_volumes[pool["token1"]["id"]] += vol / 2

        print(f"Processed batch {i} to {i+batch_size}, total poolDayData: {len(day_data)}")
        time.sleep(0.3)

    return pool_volumes


In [4]:
def main():
    print("Fetching pools created in feb 2024...")
    pools = fetch_pools_created_october()
    print(f"Total pools fetched: {len(pools)}")

    print("Extracting unique tokens from pools...")
    tokens = extract_unique_tokens_from_pools(pools)
    print(f"Total unique tokens found: {len(tokens)}")

    pool_ids = [pool["id"] for pool in pools]
    print(f"Fetching poolDayData for {len(pool_ids)} pools...")

    volumes = fetch_pool_day_data(pool_ids)

In [5]:
def main():
    print("Fetching pools created in feb 2024...")
    pools = fetch_pools_created_october()
    print(f"Total pools fetched: {len(pools)}")

    print("Extracting unique tokens from pools...")
    tokens = extract_unique_tokens_from_pools(pools)
    print(f"Total unique tokens found: {len(tokens)}")

    pool_ids = [pool["id"] for pool in pools]
    print(f"Fetching poolDayData for {len(pool_ids)} pools...")

    volumes = fetch_pool_day_data(pool_ids)
    print(f"Calculated monthly volumes for {len(volumes)} tokens.")

    # Sort tokens by volume descending
    sorted_tokens = sorted(volumes.items(), key=lambda x: x[1], reverse=True)

    print("\nTop 5 tokens by volume in feb 2024:")
    for token_id, vol in sorted_tokens[:5]:
        symbol = tokens[token_id]["symbol"] if token_id in tokens else "UNKNOWN"
        print(f"{symbol} ({token_id}): ${vol:,.2f}")

    # Now slice tokens ranked 100 to 199 (0-based indexing)
    subset_tokens = sorted_tokens[100:200]

    print("\nTop 5 tokens by volume ranked 100-200:")
    for token_id, vol in subset_tokens[:5]:
        symbol = tokens[token_id]["symbol"] if token_id in tokens else "UNKNOWN"
        print(f"{symbol} ({token_id}): ${vol:,.2f}")

    # Now slice tokens ranked 50 to 99 (0-based indexing)
    subset_tokens = sorted_tokens[50:100]

    print("\nTop 5 tokens by volume ranked 50-100:")
    for token_id, vol in subset_tokens[:5]:
        symbol = tokens[token_id]["symbol"] if token_id in tokens else "UNKNOWN"
        print(f"{symbol} ({token_id}): ${vol:,.2f}")
        
        
    # Sort tokens by volume descending
    sorted_tokens = sorted(volumes.items(), key=lambda x: x[1], reverse=True)

    print("\nTop 20 tokens by volume in feb 2024:")
    for token_id, vol in sorted_tokens[:20]:
        symbol = tokens[token_id]["symbol"] if token_id in tokens else "UNKNOWN"
        print(f"{symbol} ({token_id}): ${vol:,.2f}")


if __name__ == "__main__":
    main()
    

Fetching pools created in feb 2024...
Fetched 1000 pools, total so far: 1000
Fetched 398 pools, total so far: 1398
Total pools fetched: 1398
Extracting unique tokens from pools...
Total unique tokens found: 1308
Fetching poolDayData for 1398 pools...
Processed batch 0 to 50, total poolDayData: 296
Processed batch 50 to 100, total poolDayData: 397
Processed batch 100 to 150, total poolDayData: 248
Processed batch 150 to 200, total poolDayData: 305
Processed batch 200 to 250, total poolDayData: 354
Processed batch 250 to 300, total poolDayData: 249
Processed batch 300 to 350, total poolDayData: 190
Processed batch 350 to 400, total poolDayData: 212
Processed batch 400 to 450, total poolDayData: 156
Processed batch 450 to 500, total poolDayData: 170
Processed batch 500 to 550, total poolDayData: 212
Processed batch 550 to 600, total poolDayData: 173
Processed batch 600 to 650, total poolDayData: 221
Processed batch 650 to 700, total poolDayData: 111
Processed batch 700 to 750, total poolD